In [ ]:
# clean data
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# loai bo so, ky tu dac biet, stopwords, tim stem
def text_data(pathdata):
    with open(pathdata,'r') as f:
        text=f.read()
    text=text.lower()
    text=re.sub(r'\d+',' ',text)
    text=re.sub(r'_',' ',text)
    words=re.split(r'\W+',text)
    words=[ PorterStemmer().stem(word)
            for word in words
            if word not in stopwords.words('english') ]
    text=' '.join(words)
    return text

def collect_data(pathin):
    data=[]
    label=0
    listfolder=os.listdir(pathin)
    for folder in listfolder:
        pathdocument=pathin+"\\"+folder
        listdocument=os.listdir(pathdocument)
        for doct in listdocument:
            text=text_data(pathdocument+"\\"+doct)
            data.append(str(label)+"<<>>"+doct+"<<>>"+text)
        label+=1
    text='\n'.join(data)
    return text

text_train=collect_data("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\20news-bydate-train")
text_test=collect_data("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\20news-bydate-test")
text_full=text_train+text_test
with open("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\text_test",'w') as f:
    f.write(text_test)
with open("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\text_train",'w') as f:
    f.write(text_train)
with open("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\text_full",'w') as f:
    f.write('\n'.join(text_full))


In [ ]:
# compute feature vector
import numpy as np

# compute idf function
def compute_idf(numducument, df):
    assert df > 0
    return np.log10(numducument/df*1.)

# compute tf-idf function
def compute_tfidf(fwd,tfmax,idf):
    return idf*1.0*fwd/tfmax
# find pos with binary
def isappear(word,wordslist):
    l=0
    r=len(wordslist)-1
    while(l<=r):
        m=int((r+l)/2)
        if(word==wordslist[m]):
            return m
        else:
            if(word<wordslist[m]):
                r=m-1
            else:
                l=m+1
    return 0

# compute feature and idf
def word_feature(pathin):
    words_list = []
    cntdocument = 0
    with open(pathin, 'r') as f:
        data = f.read().splitlines()
    #find words list
    for document in data:
        text_data = document.split("<<>>")
        text = text_data[-1].split()
        words_list += list(set(text))
    words_list = list(set(words_list))
    words_list.sort()
    words_df = dict.fromkeys(words_list, 0)
    # compute df
    for document in data:
        text_data = document.split("<<>>")
        text = set(text_data[-1].split())
        for word in text:
            words_df[word] += 1
        cntdocument += 1
    # compute words_idf
    words_idf = { word : compute_idf(cntdocument, words_df[word])
                 for word in list(words_df.keys())
                 if words_df[word] > 10}
    return words_idf


# write idf function
def write_idf(words_idf):
    n=len(words_idf)
    words=list(words_idf.keys())
    idf=list(words_idf.values())
    with open("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\idf", 'w') as f:
        for i in range(n):
            f.write(words[i]+"<<>>"+str(idf[i])+"\n")


def find_feature_vector(pathdct):
    words_idf=word_feature(pathdct)
    # write idf
    write_idf(words_idf)
    feature_words=list(words_idf.keys())
    idf=list(words_idf.values())
    n=len(words_idf)
    feature_matrix=[]
    with open(pathdct,'r') as f:
        data=f.read().splitlines()
    #find feature vector
    for dct in data:
        ifo=dct.split("<<>>")
        listwords=ifo[2].split()
        setwords=set(listwords)
        words=[ word for word in setwords
                if isappear(word,feature_words)]
        tfmax=max([ listwords.count(word) for word in words ])
        dct_vector=ifo[0]+"<<>>"+ifo[1]+"<<>>"
        for word in words:
            pos=isappear(word,feature_words)
            dct_vector+=str(pos)+":"+str(compute_tfidf(listwords.count(word),tfmax,idf[pos]))+" "
        feature_matrix.append(dct_vector)
    return feature_matrix

tf_idf=find_feature_vector("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\text_train")
# write feature matrix
with open("C:\\Users\\nql\\Desktop\\20192\\project2\\textpreprocessing\\tf_idf1",'w') as f:
    f.write('\n'.join(tf_idf))
